#Import Libraries

In [0]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPool2D, MaxPool1D
from tensorflow.keras.layers import Dropout, BatchNormalization

from google.colab import drive

print("Module Loaded.")
print("TensorFlow Version :{}".format(tf.__version__))
print("NumPy Version :{}".format(np.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

TensorFlow 2.x selected.
Module Loaded.
TensorFlow Version :2.1.0-rc1
NumPy Version :1.17.4
Matplotlib Version :3.1.2


In [0]:
drive.mount('/content/drive')

data_path = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class_name = np.array([item.name for item in data_path.glob('*')])
print(class_name)

['bookshelves' 'fireplace' 'floor lamp' 'chair' 'entertainment center'
 'dressing table' 'bed' 'curtain' 'sofa' 'table']


# Prepare Data

- **Unzip Data**

In [0]:
import os

cwd = os.getcwd()
print(cwd)

!unzip /content/drive/Shared\ drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture.zip -d /content/furniture

/content
Archive:  /content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture.zip
replace /content/furniture/bed/bed (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/furniture/bed/bed (10).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Resize Image

In [0]:
# from PIL import Image

# for j in class_name[1:4]:
#     image = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data/furniture/'+j)
#     image = list(image.glob('*.jpg'))
#     for i in range(len(image)):
#         im = Image.open(str(image[i])).convert('RGB')
#         resize_image = im.resize((im.size[0]//2,im.size[1]//2))
#         resize_image.save('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture/{0}/{1}{2:03d}.jpg'.format(j,j,i))
#         print(j, i)

In [0]:
# data_path = pathlib.Path('/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/data_resize/furniture')

#ImageGenerator

In [0]:
img_size = 112
batch_n = 256

data_path = pathlib.Path('/content/furniture')

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                                width_shift_range=0.3,
                                height_shift_range=0.3,
                                horizontal_flip=True, 
                                #vertical_flip=True,
                                #rotation_range=30,
                                zoom_range=0.1,
                                brightness_range=[0.8,1.2],
                                validation_split=0.2,
                                rescale=1./255)

In [0]:
train_data_generate = image_generator.flow_from_directory(
                        directory=str(data_path),
                        batch_size=batch_n,
                        shuffle=True,
                        target_size=(img_size, img_size),
                        classes = list(class_name),
                        subset='training')
test_data_generate = image_generator.flow_from_directory(
                        directory=str(data_path),
                        batch_size=batch_n,
                        shuffle=True,
                        target_size=(img_size, img_size),
                        classes = list(class_name),
                        subset='validation')

Found 12559 images belonging to 10 classes.
Found 3135 images belonging to 10 classes.


# Load Prev Model

In [0]:
save_path = '/content/drive/Shared drives/scsa_2019_b/Project_Ai-KEA/code/model_checkpoint/'

In [0]:
# date, epoch, model_name, num
# save_model('day02', epoch=epoch_n, model_name=model_name, num='sey01')

def save_model(date, epoch, model_name, num):
    ai_kea.save(save_path+"aikea_{}_{}_{}_{}.hdf5".format(date, epoch, model_name, num))

In [0]:
def load_aikea(date, epoch, model_name, num):
  model = tf.keras.models.load_model(save_path+"aikea_{}_{}_{}_{}.hdf5".format(date, epoch, model_name, num))
  return model

In [0]:
# prev_aikea = load_aikea('day01', epoch=epoch_n, model_name=model_name, num='sey01')

#Load VGG

import model

In [0]:
import tensorflow.keras.applications.vgg16 as vgg16

model_imported = vgg16.VGG16(
    weights="imagenet", 
    include_top=False,    # dense layer 이 후는 제외
    input_tensor=tf.keras.layers.Input(shape=(img_size, img_size, 3))
    #input_tensor=keras.layers.Input(shape=(49, 49, 3))
)

model_imported.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 112, 112, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 112, 112, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 112, 112, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 56, 56, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 56, 56, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 56, 56, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 28, 28, 128)       0     

##fine tuning

freeze layers

In [0]:
# slicing which layers not to train
for layer in model_imported.layers[:-4] :
  layer.trainable = False

for layer in model_imported.layers :
  print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fe6100fe780> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe69884f6d8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe6a71fa7f0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe60e8f9780> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe6108de828> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe6105deb70> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe6105dedd8> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe6105e25c0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe6105eb908> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fe6105ebba8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fe6105f2fd0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

make ai-kea model

In [0]:
ai_kea = tf.keras.models.Sequential()

ai_kea.add(model_imported)

# Here add new layers
ai_kea.add(Flatten())
# ai_kea.add(Dense(1024, activation='relu'))
# model_fine.add(keras.layers.Dropout(0.5))
#model_fine.add(keras.layers.LeakyReLU(alpha=0.1))
# ai_kea.add(BatchNormalization())   # 필히 사용할 것
ai_kea.add(Dense(10, activation='softmax'))
ai_kea.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                46090     
Total params: 14,760,778
Trainable params: 7,125,514
Non-trainable params: 7,635,264
_________________________________________________________________


#model compile & train

In [0]:
model_name = 'vgg16' # 불러온 모델 이름
date = 'day02'
epoch_n = 100

In [0]:
def lr_schedule(epoch):
    lr = 1e-2
    if epoch/epoch_n > 0.8:
        lr *= 0.5e-3
    elif epoch/epoch_n > 0.6:
        lr *= 1e-3
    elif epoch/epoch_n > 0.4:
        lr *= 1e-2
    elif epoch/epoch_n > 0.2:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [0]:
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule(0))
lrate = LearningRateScheduler(lr_schedule)
callbacks_list = [lrate]

Learning rate:  0.01


In [0]:
ai_kea.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history = ai_kea.fit_generator(
    train_data_generate,
    epochs = epoch_n,
    validation_data = test_data_generate
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 50 steps, validate for 13 steps
Epoch 1/100
50/50 [==============================] - 97s 2s/step - loss: 2.2424 - accuracy: 0.2130 - val_loss: 1.9930 - val_accuracy: 0.2852
Epoch 2/100
50/50 [==============================] - 92s 2s/step - loss: 1.7463 - accuracy: 0.3750 - val_loss: 1.5147 - val_accuracy: 0.4517
Epoch 3/100
50/50 [==============================] - 91s 2s/step - loss: 1.3004 - accuracy: 0.5363 - val_loss: 1.3365 - val_accuracy: 0.5340
Epoch 4/100
50/50 [==============================] - 91s 2s/step - loss: 1.1087 - accuracy: 0.6157 - val_loss: 1.2309 - val_accuracy: 0.5892
Epoch 5/100
50/50 [==============================] - 91s 2s/step - loss: 0.9543 - accuracy: 0.6729 - val_loss: 1.0942 - val_accuracy: 0.6367
Epoch 6/100
50/50 [==============================] - 91s 2s/step - loss: 0.7823 - accuracy: 0.7344 - val_loss: 0.9606 - val_acc

# Save Model

In [0]:
%%time

# date, epoch, model_name, num
save_model(date, epoch=epoch_n, model_name=model_name, 'sey01')

# Plot Learning Status

In [0]:
loss = history.history['loss']
epochs = range(1, len(loss)+1)

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.title('Validation Loss')
plt.semilogy(epochs, history.history['val_loss'], 'b', label='CNN')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim([0.0, 0.6])
plt.legend(loc='best')

plt.subplot(2, 1, 2)
plt.title('Validation Accuray')
plt.semilogy(epochs, history.history['val_accuracy'], 'b', label='CNN')
plt.grid(True)
plt.ylabel('Accuracy')
plt.ylim([0.5, 0.9])
plt.legend(loc='best')
plt.show()

In [0]:
print(np.max(history.history['val_accuracy']))

> ### CAM

- make Model Function

In [0]:
import cv2
import matplotlib.patches as patches
from keras.applications.vgg16 import decode_predictions
from tensorflow.keras.preprocessing import image


def buildCAM(img_path, img_alpha, heatmap_alpha): # 0<= alpha <=1

    # load the image with the required shape
    img = image.load_img(img_path, target_size=(img_size, img_size))
    # check image
    plt.imshow(img)
    # convert the image to an array
    img = image.img_to_array(img)         # tf.keras.processing
    # expand dimensions so that it represents a single 'sample'
    img = np.expand_dims(img, axis=0)     # numpy
    # prepare the image (e.g. scale pixel values for the vgg)
    img = vgg16.preprocess_input(img)     # VGG16.preprocess_input

    grad_model = tf.keras.models.Model(
        [ai_kea.inputs],                           # input: vgg16 inputs
        [ai_kea.get_layer('conv_feat').output,  # outputs: vgg16 last conv
        ai_kea.output]                           # softmax out
    )

    # GradientTape 설정
    with tf.GradientTape() as tape:
        conv_outputs, predict = grad_model(img)   # feature, predict
        class_out = predict[:, np.argmax(predict[0])]    # class     
    print(predict[0])

    # feature map
    output = conv_outputs[0]

    # Get gradient for input image
    grads = tape.gradient(class_out, conv_outputs)[0]

    # check grads' shape
    print('grads', grads.shape, output.shape)

    # Average gradients spatially
    weights = tf.reduce_mean(grads, axis=(0, 1))

    
    # Build a map of filters according to gradients importance
    cam = np.ones(output.shape[0:2], dtype=np.float32)

    for index, w in enumerate(weights):
        print(w)
        cam += w * output[:, :, index]

    # check cam shape
    print('cam', cam.shape, weights.shape)
    print(cam)


    # image read: ndarray type
    img = cv2.imread(img_path)
    # GBR -> RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # cv2는 기본적으로 image를 GBR로 처리

    plt.imshow(img)

    print('img', img.shape, type(img))


    # resize cam
    cam = cv2.resize(cam.numpy(), (img.shape[1], img.shape[0]))
    cam = np.maximum(cam, 0)          # 0 이상의 값만 취함
    # heatmap : minmax normalized one
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())

    # convert heatmap to rgb
    heatmap = np.uint8(255 * heatmap)                       # RGB range (0~255)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)  # apply color map
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)      # colormap to RGB
    
    # get heatmap outline
    heatlist = heatmap.tolist()
    t_pos = []
    for i in range(len(heatlist)):
      for j in range(len(heatlist[i])):
        if heatlist[i][j][2]<200: t_pos.append((i, j))

    y_max = max([i[0] for i in t_pos])
    y_min = min([i[0] for i in t_pos])
    x_max = max([i[1] for i in t_pos])
    x_min = min([i[1] for i in t_pos])

    # check heatmap shape
    plt.imshow(heatmap)
    heatmap.shape #(426, 640, 3)

    # blending heatmap: image: 100%, heatmap: 50%
    output_image = cv2.addWeighted(img.astype('uint8'), img_alpha,  # 1: 100%
                                  heatmap, heatmap_alpha,            # 0.5: 50%
                                  0)
    
    
    return predict[0], output_image, y_min, x_min, (y_max-y_min), (x_max-x_min)

In [0]:
img_path = '/content/furniture/table/table1'

In [0]:
import cv2
import matplotlib.patches as patches
from keras.applications.vgg16 import decode_predictions
from tensorflow.keras.preprocessing import image

# define Model
pred_list=[]
pred_list, output_image, yy, xx, h, w = buildCAM(img_path, 1, 0)
pred_cls = class_name[np.argmax(pred_list)]
print(pred_cls)
fig, ax = plt.subplots(1)
rect = patches.Rectangle((xx, yy), w, h, linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(rect)

rect_txt = patches.Rectangle((xx,yy), 80, 20, edgecolor='r', facecolor='r')
ax.add_patch(rect)
ax.add_patch(rect_txt)
ax.text(xx+10, yy+15, 
         '{}'.format(pred_cls),
         fontsize=18, color='w')
ax.imshow(output_image)